In [1]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2025):
    with open('sessionID') as f:
        sessionId = f.readlines()[0]
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data



In [2]:
data = [\
'3-5',
'10-14',
'16-20',
'12-18',
'4-7',
'',
'1',
'5',
'8',
'11',
'17',
'32']
data = get_data(5)



In [3]:
import itertools
records = [list(y) for x, y in itertools.groupby(data, lambda z: z == '') if not x]


In [4]:
ranges = []
for line in records[0]:
    a,b = line.split('-')
    ranges.append((int(a), int(b)))
ids = list(map(int,(records[1])))

In [22]:
def merge_ranges(ranges):
    """
    Merges overlapping or adjacent integer ranges into a minimal set of non-overlapping tuples.

    Args:
        ranges: A list of tuples, where each tuple is (start, end).

    Returns:
        A new list of merged tuples.
    """
    if not ranges:
        return []

    # 1. Sort the ranges by their start value
    # This is crucial for the merging logic to work
    sorted_ranges = sorted(ranges, key=lambda x: x[0])

    merged = []
    # Start with the first range as the current range to compare against
    current_start, current_end = sorted_ranges[0]

    # 2. Iterate through the rest of the sorted ranges
    for next_start, next_end in sorted_ranges[1:]:
        # Check if the next range overlaps with or is adjacent to the current range
        # An overlap/adjacency occurs if the next range starts at or before 
        # the current range ends (or one step beyond if we consider adjacency).
        # We assume start <= end for all ranges.

        if next_start <= current_end + 1:
            # There is an overlap or adjacency, so we extend the current range's end
            # We take the maximum end point so the merged range covers both
            current_end = max(current_end, next_end)
        else:
            # No overlap/adjacency, the current merged range is complete.
            # Add it to the result list.
            merged.append((current_start, current_end))
            # Start a new current range with the next range's values
            current_start, current_end = next_start, next_end

    # 3. Append the very last current range after the loop finishes
    merged.append((current_start, current_end))

    return merged

def find_values_in_ranges_optimized(merged_ranges, values_to_check):
    """
    Optimized function to filter a sorted list of values against a sorted list 
    of non-overlapping merged ranges using a two-pointer approach.

    Args:
        merged_ranges: A list of non-overlapping, sorted (start, end) tuples.
        values_to_check: A list of integers (which should be sorted before calling).

    Returns:
        A list of values that fall within the ranges.
    """
    # Ensure values are sorted if they aren't already. Ranges are assumed sorted from previous step.
    sorted_values = sorted(values_to_check)
    
    results = []
    range_index = 0
    num_ranges = len(merged_ranges)

    # Iterate through the sorted values list
    for value in sorted_values:
        merged_ranges[range_index][1]
        # Advance the range pointer until we find a range that the current value 
        # might possibly fall into (i.e., the range's end is >= the value)
        while range_index < num_ranges and merged_ranges[range_index][1] < value:
            range_index += 1
            
        # If we ran out of ranges to check, no future values will match either
        if range_index >= num_ranges:
            break
            
        # Check if the value falls within the current range's start and end
        current_start, current_end = merged_ranges[range_index]
        if current_start <= value <= current_end:
            results.append(value)
            
        # Optional: If you know ranges are strictly non-overlapping, 
        # we don't need to check further ranges for this specific value (the loop handles this).

    return results

def calculate_total_count(merged_ranges):
    """
    Calculates the total count of individual integers covered by a list 
    of non-overlapping (start, end) tuples.

    Args:
        merged_ranges: A list of non-overlapping (start, end) tuples.

    Returns:
        The total count of covered integers.
    """
    total = 0
    # Iterate through each non-overlapping range
    for start, end in merged_ranges:
        # The number of integers in a range inclusive of both ends is (end - start + 1)
        total += (end - start + 1)
        
    return total

In [24]:
merged_ranges = merge_ranges(ranges)
print('part 1: ',len(find_values_in_ranges_optimized(merge_ranges(ranges), ids)))
print('part 2: ',calculate_total_count(merged_ranges))

part 1:  563
part 2:  338693411431456
